In [1]:
# external package imports
import numpy as np
import pandas as pd
import os, pdb, glob, time, argparse
from os.path import exists, split, isdir, getsize

# internal package imports
from sdo_clv_pipeline.paths import root
from sdo_clv_pipeline.sdo_io import *
from sdo_clv_pipeline.sdo_process import *

ModuleNotFoundError: No module named 'sdo_clv_pipeline'

## Preliminaries
First we need to find our files and then read them in. They are plain FITS files that can be read with astropy or sunpy, but I wrote some extra code to help parse out the data for our purposes. If you're curious, you can look in ```sdo_image.py```

In [ ]:
# define the paths where files live
con_file = "/Users/mpalumbo/Downloads/hmi.Ic_720s.20140107_000000_TAI.continuum.fits"
mag_file = "/Users/mpalumbo/Downloads/hmi.M_720s.20140107_000000_TAI.magnetogram.fits"
dop_file = "/Users/mpalumbo/Downloads/hmi.V_720s.20140107_000000_TAI.Dopplergram.fits"
aia_file = "/Users/mpalumbo/Downloads/aia_lev1_1700a_2014_01_07t00_00_30_71z_image_lev1.fits"

In [ ]:
# create the SDOImage object instances
con = SDOImage(con_file)
mag = SDOImage(mag_file)
dop = SDOImage(dop_file)
aia = SDOImage(aia_file)

## Initial Processing
Next we need figure out the coordinates of our images and "correct" for some physical effects. Initially our images are all in pixel coordinates, but the FITS header has the information we need to transform from pixel coordinates to a heliophysical coordinate systems. If you want to go down the rabit whole of coordinate systems, you can look here: https://www.aanda.org/articles/aa/pdf/2006/14/aa4262-05.pdf

In [ ]:
# do the coordinate transforms, etc.
dop.calc_geometry()
con.inherit_geometry(dop)
mag.inherit_geometry(dop)

In [ ]:
# calculate the limb darkening correction
con.calc_limb_darkening()
aia.calc_limb_darkening()

In [ ]:
# correct magnetogram for foreshortening
mag.correct_magnetogram()

In [ ]:
# calculate differential rot., meridional circ., obs. vel, grav. redshift, cbs
# dop.correct_dopplergram(fit_cbs=False)

In [ ]:
# set values to nan for mu less than mu_thresh
mu_thresh = 0.1
con.mask_low_mu(mu_thresh)
dop.mask_low_mu(mu_thresh)
mag.mask_low_mu(mu_thresh)
aia.mask_low_mu(mu_thresh)

## Identifying Pixels
The ```SunMask``` constructor performs the task of indentifying pixels. You'll be playing with/augmenting the code in this function! 

In [ ]:
mask = SunMask(con, mag, dop, aia)
mask.mask_low_mu(mu_thresh)

## Plotting
I have some plotting code that you can use to visualize the images we processed

In [ ]:
plot_image(con)

: 